In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Linear combination

In [2]:
# lets implement lienar combination in vectorized form

def linear_combination(X:np.ndarray, w: np.ndarray) -> np.ndarray:
    '''Calculate linear combination of features.
    
    The linear combination is calculated with the following vectorized form
    z = Xw
    
    Args:
        X: feature matrix with shape(n,m)
        w: weight vector with shape(m,)
        
    Returns:
        Linear combination of feature with (n,)
        
    '''
    return X@w

In [3]:
# let's implement sigmoid function in  a vectorized form

def sigmoid(z: np.ndarray) -> np.ndarray:
    '''Calculates sigmoid of linear combination of features z.
    
    Args:
        z: list of floats
        
    Returns:
        List of output of sigmoid function
    '''
    return 1/(1+np.exp(-z))



### if activation or probability > threshold then we label the sample with class 1 or 0

In [4]:
def predict(X:np.ndarray, w:np.ndarray, threshold: float)-> np.ndarray:
    '''Predicts class label for samples.
    
    The samples are represented with a bunch of features and are presented in form of a freature matrix. The class label is predicted as follows:
    * If sigmoid(Xw) > threshold, the sample is labeled with class 1.
        * else class 0
        
    Args:
        X: feature vector of shape(n,m)
        w: weight vector of shape(m,)
        threshold: probability threshold for classification
        
    Returns:
        A list of class labels of shape (n,)
    '''
    return np.where(sigmoid(linear_combination(X,w)) > threshold, 1, 0)

## applying the model

In [5]:
feature_matrix = np.array([[1,20,2], [1,2,2]])
weight_vector = np.array([-1,0,1])

print('Shape of feature matrix: ', feature_matrix.shape)
print('Shape of weight vector: ', weight_vector.shape)

class_labels = predict(feature_matrix, weight_vector, 0.5)

print('Shape of output: ', class_labels.shape)
print('The class label vector is: ', class_labels)


Shape of feature matrix:  (2, 3)
Shape of weight vector:  (3,)
Shape of output:  (2,)
The class label vector is:  [1 1]


# loss function

In [6]:
def loss(y, sigmoid_vectors, weight_vector, l1_reg_rate, l2_reg_rate):
    return( -1 * (np.sum(y * np.log(sigmoid_vector) + (1-y) * np.log(1-sigmoid_vector)))
          + l2_reg_rate * np.dot(np.transpose(weight_vector), weight_vector)
          + l1_reg_rate * np.sum(np.abs(weight_vector)) )

In [7]:
def calculate_gradient(X:np.ndarray, y:np.ndarray, w:np.ndarray, reg_rate:float) -> np.ndarray:
    '''Calculates gradients of loss function w.r.t. weight vector on training set.
    The gradient is claculated with the following vectorized operation:
        np.transpose(X)(sigmoid(Xw) -y) + \lambda w
    
    Args:
        X: Feature matrix for training data.
        y: Label vector for training data.
        reg_rate: regularization rate
        
    Returns:
        A vector of gradients.
    '''
    
    return np.transpose(X) @ (sigmoid(linear_combination(X,w)) - y) + reg_rate*w

# Logistic regression class implementation

In [8]:
class LogisticRegression(object):
    '''Logistic regression model.
    y = sigmoid(X @ w)
    '''
    def set_weight_vector(self, w):
        self.w = w
    
    def linear_combination(self, X:np.ndarray) -> np.ndarray:
        '''Calculates linear combination of features.
        The linear combination is calculated with the following vectorized from:
            z = Xw
            
        Args:
            X: feature matrix with shape(n,m)
        
        Returns:
            Linear combination of features with shape(n,)
        '''
        return X@self.w
    
    def sigmoid(self, z: np.ndarray):
        '''Return probability of input belonging class 1.
        Args:
            z : (n, ) np.ndarray
        Returns:
            sigmoid activation vector (n,) np.ndarray
        '''
        return 1/(1 + np.exp(-z))
    
    def activation(self, X:np.ndarray) -> np.ndarray:
        '''Calculates sigmoid activation for logistic regression.
        
        The sigmoid activation is calculated with the following vectorized form:
            act = sigmoid(Xw)
            
        Args:
            X: feature matrix with shape (n,m)
            
        Returns:
            activation vector with shape (n,)
        '''
        return self.sigmoid(self.linear_combination(X))
    def predict(self, x: np.ndarray, threshold: float = 0.5):
        '''Classify input data.
        Args:
            x; (N,D) np.ndarray
            threshold : float, optional
                threshold of binary classification (default is 0.5)
        
        Returns:
            (N, ) np.ndarray
        '''
        return (self.activation(x) > threshold).astype(int)
    def loss(self, X:np.ndarray, y: np.ndarray, reg_rate: float) -> float:
        ''' Calculates binary cross entropy loss on training set.
        
        Args:
            X: Feature matrix for training data.
            y: Label vector for training data.
            reg_rate: regularization rate
            
        Returns:
            loss.
        '''
        
        predicted_prob = self.activation(X)
        return (-1 * (np.sum(y * np.log(predicted_prob) + (1-y) *
                            np.log(1-predicted_prob))) + 
                           reg_rate * np.dot(np.transpose(self.w), self.w))
    
    def calculate_gradient(self, x: np.ndarray, y: np.ndarray,
                              reg_rate:float) -> np.ndarray:
        '''Calculates gradients of loss function w.r.t. weight vector on training set.
        
        Args:
            X: Feature matrix for training data.
            y: Label vector for training data.
            reg_rate: regularization rate
        
        Returns:
            A vector of gradients.
        '''
        return np.transpose(X)@(self.activation(X) -y) + reg_rate * self.w
    
    def update_weight(self, grad:np.ndarray, lr:float) -> np.ndarray:
        '''Updates the weights based on the gradient of loss function.
        
        Weight updates are carried out with the following formula:
            w_new := w_old - lr* grad
        
        Args:
            grad: gradient of loss w.r.t w
            lr: learning rate
        
        Returns:
            Updated weight vector
        '''
        return (self.w - lr*grad)
    def gd(self, X:np.ndarray, y:np.ndarray,
              num_epochs:int, lr:float, reg_rate:float) -> np.ndarray:
        '''Estimates parameters of linear regression model through gradient descent.
        
        Args:
            X: Feature matrix for training data.
            y: Label vector for training data.
            num_epochs: Number of training steps
            lr: Learning rate
            reg_rate: regularization rate
        
        Return:
            Weight vector: Final weight vector
        '''
        self.w = np.zeros(X.shape[1])

# Logistic Regression demonstration

In [20]:
import numpy as np
import random

In [10]:
# creating a toy data set creaing function

def create_toy_data():
    x0 = np.random.normal(size=50).reshape(-1,2) - 1
    x1 = np.random.normal(size=50).reshape(-1,2) +1
    return np.concatenate([x0,x1]), np.concatenate([np.zeros(shape=50)])

In [11]:
feature_matrix, label_vector = create_toy_data()
print('Shape of feature matrix: ', feature_matrix.shape)
print('sahpe of label vector: ', label_vector.shape)

Shape of feature matrix:  (50, 2)
sahpe of label vector:  (50,)


## Polynomial transfromation

In [24]:
from ml_models import *

In [26]:
feature_matrix_bias = polynomial_transform(feature_matrix, degree=1)

NameError: name 'np' is not defined